# Estimate the known vs novel credbile sets

### Download the dataset of known vs novel studies

In [3]:
!gcloud storage cp -r gs://genetics-portal-dev-analysis/yt4/20250403_for_gentropy_paper/known_studyLocusIds ../../data/.


Copying gs://genetics-portal-dev-analysis/yt4/20250403_for_gentropy_paper/known_studyLocusIds/_SUCCESS to file://../../data/known_studyLocusIds/_SUCCESS
Copying gs://genetics-portal-dev-analysis/yt4/20250403_for_gentropy_paper/known_studyLocusIds/part-00000-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet to file://../../data/known_studyLocusIds/part-00000-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet
Copying gs://genetics-portal-dev-analysis/yt4/20250403_for_gentropy_paper/known_studyLocusIds/part-00001-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet to file://../../data/known_studyLocusIds/part-00001-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet
Copying gs://genetics-portal-dev-analysis/yt4/20250403_for_gentropy_paper/known_studyLocusIds/part-00002-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet to file://../../data/known_studyLocusIds/part-00002-2f0f4342-2a7e-4b1e-b6e0-8883049cb9a4-c000.snappy.parquet
Copying gs://genetics-portal-dev-a

In [9]:
from gentropy.common.session import Session
from pyspark.sql import DataFrame
from pyspark.sql import functions as f


In [10]:
session = Session(extended_spark_conf={"spark.driver.memory": "40G"})
variant_index_path = "../../data/variant"
study_index_path = "../../data/study"
credible_set_path = "../../data/credible_set"
known_studies_path = "../../data/known_studyLocusIds"


In [11]:
ks = session.spark.read.parquet(known_studies_path)
cs = session.spark.read.parquet(credible_set_path)


In [21]:
def classify_study_locus(all: DataFrame, known: DataFrame) -> DataFrame:
    """Classify study locus ids based on known or novel."""
    assert "studyLocusId" in all.columns and "studyLocusId" in known.columns
    known = known.withColumn("studyLocusClass", f.lit("known")).alias("r")
    classified = all.alias("l").join(known, on="studyLocusId", how="left")
    return classified.withColumn(
        "studyLocusClass", f.when(f.col("studyLocusClass") == "known", f.lit("known")).otherwise("novel")
    )


In [ ]:
classified_cs = classify_study_locus(cs, ks)
classified_cs.groupBy("studyLocusClass").agg(f.count("studyLocusId").alias("count")).select(
    f.col("studyLocusClass"),
    f.col("count"),
    f.round((f.col("count") / classified_cs.count() * f.lit(100)), 2).alias("percent"),
).show()


AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [studyLocusClass#1010, count#1068L, round(((cast(count#1068L as double) / cast(sum(count#1068L) as double)) * cast(100 as double)), 2) AS percent#1072]
+- Aggregate [studyLocusClass#1010], [studyLocusClass#1010, count(studyLocusId#4) AS count#1068L]
   +- Project [studyLocusId#4, studyId#5, variantId#6, chromosome#7, position#8, region#9, beta#10, zScore#11, pValueMantissa#12, pValueExponent#13, effectAlleleFrequencyFromSource#14, standardError#15, subStudyDescription#16, qualityControls#17, finemappingMethod#18, credibleSetIndex#19, credibleSetlog10BF#20, purityMeanR2#21, purityMinR2#22, locusStart#23, locusEnd#24, sampleSize#25, ldSet#26, locus#27, ... 4 more fields]
      +- Project [studyLocusId#4, studyId#5, variantId#6, chromosome#7, position#8, region#9, beta#10, zScore#11, pValueMantissa#12, pValueExponent#13, effec